In [3]:
import glob
import torch

from model import load_untrained_bert_base
from dataset_loading import load_tokenized_dataset, load_glue_sentence_classification
from finetune import run_epoch
from pathlib import Path

In [4]:
dataset = load_tokenized_dataset(
    Path("../../GLUE-baselines/glue_data/SST-2/"),
    load_glue_sentence_classification
)

In [8]:
from transformers import AutoModelForSequenceClassification
from model import get_bert_config

def print_checkpoints(path):
    checkpoints = sorted(glob.glob(str(path) + "*"))
    for path in checkpoints:
        checkpoint = torch.load(path)
        print(f"Epoch {checkpoint['epochs']}:")
        print("Train loss:",checkpoint["train_loss"], "\t\tTrain accuracy:",checkpoint["train_accuracy"])
        print("Test loss:",checkpoint["test_loss"], "\t\tTest accuracy:",checkpoint["test_accuracy"])

        print(checkpoint["optimizer_state_dict"]["param_groups"][0]["lr"])

        # if checkpoint['epochs']==3:
        #     model = AutoModelForSequenceClassification.from_config(get_bert_config("base"))

        #     #print(checkpoint["model_state_dict"].keys())
        #     model.load_state_dict(checkpoint["model_state_dict"])

        #     torch.save(model,"../../models/finetuned_bert_base_ss2_92.4%.pt")
        

In [9]:
print_checkpoints("../../checkpoints/kd_finetune_notpretrained/")

Epoch 1:
Train loss: tensor(0.0626) 		Train accuracy: tensor(0.5513, device='cuda:0')
Test loss: tensor(0.0341) 		Test accuracy: tensor(0.5367, device='cuda:0')
2e-05
Epoch 2:
Train loss: tensor(0.0520) 		Train accuracy: tensor(0.5552, device='cuda:0')
Test loss: tensor(0.0312) 		Test accuracy: tensor(0.5367, device='cuda:0')
2e-05
Epoch 3:
Train loss: tensor(0.0498) 		Train accuracy: tensor(0.5552, device='cuda:0')
Test loss: tensor(0.0296) 		Test accuracy: tensor(0.5367, device='cuda:0')
2e-05
Epoch 4:
Train loss: tensor(0.0484) 		Train accuracy: tensor(0.5552, device='cuda:0')
Test loss: tensor(0.0281) 		Test accuracy: tensor(0.5367, device='cuda:0')
2e-05
Epoch 5:
Train loss: tensor(0.0474) 		Train accuracy: tensor(0.6446, device='cuda:0')
Test loss: tensor(0.0269) 		Test accuracy: tensor(0.7477, device='cuda:0')
2e-05
Epoch 6:
Train loss: tensor(0.0467) 		Train accuracy: tensor(0.8437, device='cuda:0')
Test loss: tensor(0.0260) 		Test accuracy: tensor(0.7569, device='cuda:0')
2e-0

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

: 